In [ ]:
from query.models import Video
from esper.prelude import *
import cv2
import os
import datetime

# Import videos

In [ ]:
def import_video(path):
    cap = cv2.VideoCapture(path)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    if 'CS' in path:
        m, d, y = path.split('_')[2:5]
        time = datetime.datetime(int(y), int(m), int(d))
    else:
        time = datetime.datetime(int(2019), int(1), int(1))
    print(path, num_frames, fps, width, height, time)
    Video(path=path, num_frames=num_frames, fps=fps, width=width, height=height, time=time).save()
    return Video.objects.filter(path=path)[0]

def import_frame(video):
    Frame.objects.bulk_create([Frame(number=i, video=video) for i in range(video.num_frames)])

## From folder

In [ ]:
for video_name in sorted(os.listdir('/app/data/videos/')):
    if video_name[0] == '.':
        continue
    videos = Video.objects.filter(path__contains=video_name)
    if len(videos) > 0:
        continue
    print(video_name)
    video = import_video(os.path.join('/app/data/videos/', video_name))
    import_frame(video)

## Single video

In [ ]:
video = import_video('/app/data/videos/wimbledon_2019_mens_semifinal_federer_nadal.mp4')
import_frame(video)

In [ ]:
video = Video.objects.filter(path__contains='wimbledon')[0]
video

# Process labeled hit

In [ ]:
import pickle
def parse_filename_to_fid(file_name):
    time_stamp = file_name.split('.')[2]
    h, m, s, f = time_stamp.split('_')
    fid = int(h) * 60*60*fps + int(m) * 60*fps + int(s) * fps + int(f) + 1 # premier has offset 1
    return fid

## Tennis

In [ ]:
fps = 25
file_list = pickle.load(open('/app/data/pkl/fednad.pkl', 'rb'))
hit_list = []
for file_name in file_list:
    if not file_name[0] == 'w':
        continue
    fid = parse_filename_to_fid(file_name)
    hit_list.append(fid)
hit_list.sort()

In [ ]:
hit_annot = {}
point_gap = 150
points = []
last_fid = hit_list[0]
hits = [last_fid]
for fid in hit_list[1:]:
    if fid - last_fid > point_gap:
        points.append(hits)
        hits = [fid]
        last_fid = fid
    else:
        hits.append(fid)
        last_fid = fid
pickle.dump({video.item_name() + '.mp4' : points}, open('../data/pkl/hit_annotation_tennis.pkl', 'wb'))

In [ ]:
fid = 204882 
fps = 25
h, m, s = fid / fps / 60 / 60, fid / fps // 60 % 60, fid // fps % 60
print(h, m, s)

In [ ]:
len([hit for hit in points if len(hit) > 1])

## Table tennis

In [ ]:
# manually segment video as two player changes position
# foreback_split = {(0, 15): 'CC', (15, 21): 'HW', (21, 31): 'CC',  (31, 40): 'HW', (40, 52): 'CC'} # semi2
# foreback_split = {(0, 11): 'HW', (11, 17): 'KS', (17, 23): 'HW',  (23, 32): 'KS'} # quater4
# foreback_split = {(0, 16): 'OD', (16, 23): 'JZ', (23, 23): 'OD',  (31, 42): 'JZ', (42, 53): 'OD'} # semi1
foreback_split = {(0, 12): 'TJ', (12, 19): 'JZ', (19, 26): 'TJ',  (26, 36): 'JZ', (36, 44): 'TJ'} # quarter2

In [ ]:
# collect hits from file name of snapshots
file_list = os.listdir('/app/data/label/quarterfinal_2/')
hit_list = []
fps = int(video.fps)
for file_name in file_list:
    if not file_name[0] == 'T':
        continue
    fid = parse_filename_to_fid(file_name)
    hit_list.append(fid)
hit_list.sort()
print('Totally %d hits in the video' % len(hit_list))

In [ ]:
# group hits into potins
cur_fid = hit_list[0]
hit_in_point = [[cur_fid]]
for fid in hit_list[1:]:
    if fid - cur_fid < 100:
        hit_in_point[-1].append(fid)
    else:
        hit_in_point.append([fid])
    cur_fid = fid
print('Totally %d points in the video' % len(hit_in_point))

In [ ]:
# for each point, select who gives the first hit
point_ism = IntervalSetMapping({video_id: IntervalSet([Interval(Bounds3D(h[0], h[-1])) for h in hit_in_point])})
widget = IntervalSetMapping_to_vgrid(IntervalSetMapping_frame_to_second(point_ism), flat=True)
widget

In [ ]:
# store who hits the first in each point
selection = widget.label_state['blocks_selected']
pickle.dump(selection, open('/app/result/pkl/label_fg.pkl', 'wb'))

In [ ]:
# propagate whose hit from the first hit of each point
H, W = video.height, video.width
hit_dict = []
for i, point in enumerate(hit_in_point):
    hit_dict.append([])
    for j, fid in enumerate(point):
#         person_fg, person_bg = get_densepose_by_fid(sc, video.item_name(), fid)
        is_fg = not np.logical_xor(str(i) in selection, j % 2 == 0)
#         person = person_fg if is_fg else person_bg
#         if not person is None:
#             if person.keyp[2, Person.RWrist] > Person.KP_THRESH:
#                 pos = player.keyp[:2, Person.RWrist].astype(int)
#             else:
#                 mid_shoulder = (person.keyp[:2, Person.RShoulder] + player.keyp[:2, Person.LShoulder]) // 2
#                 pos = mid_shoulder
#             pos = (int(pos[0]), int(pos[1]))
#         else:
#             pos = None
        hit_dict[-1].append({'fid': fid, 'fg': is_fg})

In [ ]:
# save data
hit_dict_split = {'JZ': [], 'TJ': []}
for point in hit_dict:
    m = point[0]['fid'] / video.fps / 60
    for (s, e) in foreback_split.keys():
        if m > s and m < e:
            hit_dict_split[foreback_split[(s, e)]].append(point)
            break

In [ ]:
hit_dict_all = pickle.load(open('/app/data/pkl/hit_annotation.pkl', 'rb'))
hit_dict_all[video.item_name()+'.mp4'] = hit_dict_split
pickle.dump(hit_dict_all, open('/app/data/pkl/hit_annotation.pkl', 'wb'))

In [ ]:
# spot check for labeled hits
point = random.choice(hit_dict_split['JZ'])
hit = random.choice(point)
# hit = hit_dict_split['JZ'][3][0]
frame = load_frame(video, hit['fid'], [])
if hit['fg']:
    cv2.circle(frame, (0, 0), 20, (0, 0, 255), -1)
else:
    cv2.circle(frame, (0, 0), 20, (0, 255, 0), -1)
imshow(frame)
print(hit)

# Export to racket2game database

In [ ]:
from scannerpy import Client, DeviceType
from scannerpy.storage import NamedVideoStream, NamedStream
from query.models import Video
from scannertools import maskrcnn_detection, densepose_detection

from esper.table_tennis.utils import *
# from esper.table_tennis.pose_utils import *
from esper.widget import *

import os
import sys
import numpy as np
import pickle

## Tennis

In [ ]:
hit_annotation = pickle.load(open('/app/data/pkl/hit_annotation_tennis.pkl', 'rb'))
frame_ids_dict = {}
for k, v in hit_annotation.items():
    hit_dict = []
#     for h in v.values():
#         hit_dict += h
    hit_dict = v
    frame_ids = [i for point in hit_dict for i in range(point[0] - 25, point[-1] + 25) ]
    frame_ids.sort()
    frame_ids_dict[k] = frame_ids

In [ ]:
# Export video table
videos_new = [
 'wimbledon_2019_mens_semifinal_federer_nadal.mp4',
]
video_table = pickle.load(open('/app/data/db/video_table.pkl', 'rb'))

for idx, video_name in enumerate(videos_new):
    video = Video.objects.filter(path__contains=video_name)[0]
    video_id = idx + len(video_table)
    video_table.append({'id': video_id, 'name': video_name, 'num_frames': video.num_frames, 
                        'width': video.width, 'height': video.height, 'fps': video.fps})
    if video_name in frame_ids_dict:
        video_table[-1]['valid_frames'] = frame_ids_dict[video_name]
pickle.dump(video_table, open('/app/data/db/video_table.pkl', 'wb'))
# video_table

In [ ]:
def update_person(person):
    mask = mask_util.decode(person.mask)
    bbox = person.bbox
    person.mask = mask_util.encode(np.asfortranarray(mask[bbox[1] : bbox[3], bbox[0] : bbox[2]]))
    return person

def get_densepose_player_from_raw(self, video_id, frame_id, foreground=None):
    if not frame_id in self.densepose_table[video_id]:
        return None
    people = self.densepose_table[video_id][frame_id]
    if len(people) < 2:
        return None

    bbox_area = []
    for person in people:
        area = (person['bbox'][2] - person['bbox'][0]) * (person['bbox'][3] - person['bbox'][1])
        bbox_area.append(area)
    top2 = np.argsort(bbox_area)[-2:]
    personA = Person(people[top2[0]])
    personB = Person(people[top2[1]])
    if personA.keyp[1, Person.LShoulder] >= personB.keyp[1, Person.LShoulder]:
        person_fg = personA
        person_bg = personB 
    else:
        person_fg = personB
        person_bg = personA 
    if foreground is None:
        return person_fg, person_bg
    elif foreground:
        return person_fg
    else:
        return person_bg

In [ ]:
# Export densepose database
sc = Client()

In [ ]:
for video_name in videos_new:
    video_id = 
    densepose_stream = NamedStream(sc, video_name[:-4] + '_densepose')
    seq = sc.sequence(densepose_stream._name)
    result = {}
    for i, obj in tqdm(enumerate(seq.load(workers=1))):
        fid = frame_ids_dict[video_name][i]
#         person_fg, person_bg = get_densepose_player_from_raw(obj) ### modify
#         if not person_fg is None:
#             result[fid] = [update_person(person_fg), update_person(person_bg)]
#         else:
#             result[fid] = []
        result[fid] = obj
    pickle.dump(result, open('/app/data/db/densepose/{:03}.pkl'.format(video.id), 'wb'))

## Table tennis

In [ ]:
hit_annotation = pickle.load(open('/app/data/pkl/hit_annotation.pkl', 'rb'))
frame_ids_dict = {}
for k, v in hit_annotation.items():
    hit_dict = []
    for h in v.values():
        hit_dict += h
    frame_ids = [i for point in hit_dict for i in range(point[0]['fid']-25, point[-1]['fid']+25) ]
    frame_ids.sort()
    frame_ids_dict[k] = frame_ids

In [ ]:
# semi1: JZ, OD
# semi2: HW, CC
# quarter2: JZ, TJ
# quarter4: HW, KS

In [ ]:
# Export video table
videos_all = [
 'Tabletennis_2012_Olympics_men_single_final_gold.mp4',
 'Tabletennis_2012_Olympics_men_single_final_bronze.mp4',
 'Tabletennis_2012_Olympics_men_single_semi_final_1.mp4',
 'Tabletennis_2012_Olympics_men_single_semi_final_2.mp4',
 'Tabletennis_2012_Olympics_men_single_quarter_final_1.mp4',
 'Tabletennis_2012_Olympics_men_single_quarter_final_2.mp4',
 'Tabletennis_2012_Olympics_men_single_quarter_final_3.mp4',
 'Tabletennis_2012_Olympics_men_single_quarter_final_4.mp4',
]
video_table = []
for idx, video_name in enumerate(videos_all):
    video = Video.objects.filter(path__contains=video_name)[0]
    video_table.append({'id': idx, 'name': video_name, 'num_frames': video.num_frames, 
                        'width': video.width, 'height': video.height, 'fps': video.fps})
    if video_name in frame_ids_dict:
        video_table[-1]['valid_frames'] = frame_ids_dict[video_name]
pickle.dump(video_table, open('/app/data/db/video_table.pkl', 'wb'))
video_table

In [ ]:
videos_new = [
#     'Tabletennis_2012_Olympics_men_single_final_gold.mp4',
#     'Tabletennis_2012_Olympics_men_single_semi_final_2.mp4',
#     'Tabletennis_2012_Olympics_men_single_quarter_final_4.mp4'
    'Tabletennis_2012_Olympics_men_single_semi_final_1.mp4',
    'Tabletennis_2012_Olympics_men_single_quarter_final_2.mp4',
]
sc = Client()

In [ ]:
# Export densepose database
for video_name in videos_new:
    video_id = videos_all.index(video_name)
    densepose_stream = NamedStream(sc, video_name[:-4] + '_densepose')
    seq = sc.sequence(densepose_stream._name)
    result = {}
    for i, obj in tqdm(enumerate(seq.load(workers=1))):
        result[frame_ids_dict[video_name][i]] = obj
    pickle.dump(result, open('/app/data/db/densepose/{:03}.pkl'.format(video_id), 'wb'))

In [ ]:
# Export hit annotation
H, W = video.height, video.width
for video_name in videos_new:
    video_id = videos_all.index(video_name)
    hit_dict = hit_annotation[video_name]
    for _, hit_split in hit_dict.items():
        for pid, point in enumerate(hit_split):
            for hid, hit in enumerate(point):
                person_fg, person_bg = get_densepose_by_fid(sc, video_name, hit['fid'])
                person = person_fg if hit['fg'] else person_bg
                if not person is None:
                    if person.keyp[2, Person.RWrist] > Person.KP_THRESH:
                        pos = person.keyp[:2, Person.RWrist].astype(int)
                    else:
                        mid_shoulder = (person.keyp[:2, Person.RShoulder] + person.keyp[:2, Person.LShoulder]) // 2
                        pos = mid_shoulder
                    pos = (int(pos[0]), int(pos[1]))
                else:
                    pos = None
                hit_split[pid][hid]['pos'] = pos
            
    json.dump(hit_dict, open('/app/data/db/hit_annotation/{:03}.json'.format(video_id), 'w'))